# 使用 Amazon Comprehend 进行情感分析

您可以使用 Amazon Comprehend 来确定文档的情绪。 您可以确定情绪是积极的、消极的、中性的还是混合的。 例如，您可以使用情感分析来确定对博客帖子的情感，以确定您的读者是否喜欢该帖子。

可以使用 Amazon Comprehend 支持的任何主要语言执行该操作。 所有文件必须使用相同的语言。

您可以使用以下任何操作来检测文档或一组文档的情绪。 

    DetectSentiment

    BatchDetectSentiment

    StartSentimentDetectionJob

这些操作返回文本最可能的情绪以及每个情绪的分数。 该分数表示正确检测到情绪的可能性。 例如，在下面的示例中，文本具有正面情绪的可能性为 95%。文本具有负面情绪的可能性不到 1%。您可以使用 SentimentScore 来确定检测的准确性是否满足您的应用程序的需求。

DetectSentiment 操作返回一个包含检测到的情绪的对象和一个 SentimentScore 对象。 BatchDetectSentiment 操作返回一个情绪列表和每个情绪对应的SentimentScore 对象。 StartSentimentDetectionJob 操作启动一个异步作业，该作业生成一个包含情绪的列表和对应的 SentimentScore分数。



本实验包括使用 Amazon Comprehend 执行情绪分析的步骤说明。 

## 初始化

首先需要指定：

* AWS 区域。
* IAM 角色 arn 用于授予对 Comprehend API 和 S3 存储桶的访问权限。 
* 您要用于存放训练数据的 S3 存储桶。 


In [ ]:
# import libraries

import os
import boto3
import re
import json

In [ ]:
# verify if bucket exist

sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
print("Your account id is {}".format(account_id))

bucket = "comprehend-labs" + account_id +  "-2"
print ("Bucket name used is " + bucket)
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

if (s3.Bucket(bucket).creation_date is None):
    s3_client.create_bucket(Bucket=bucket)
    print ("Created bucket " + bucket)
else:
    print ("Bucket Exists")

In [ ]:
prefix = "sentiment-analysis"
bucketuri="s3://"+bucket+"/"+prefix
print(bucketuri)

## 准备数据
我们首先将数据集上传到 s3 存储桶。样本数据集来自“Amazon reviews - Full”中提取的亚马逊评论，该数据集在"Character-level Convolutional Networks for Text Classification" (Xiang Zhang et al., 2015)一文当中被使用。

现在，使用Pandas读取数据并进行查看。

In [ ]:
# Download the data set

!wget https://docs.aws.amazon.com/comprehend/latest/dg/samples/tutorial-reviews-data.zip
!apt-get install unzip -y
!unzip -o tutorial-reviews-data.zip



In [ ]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd 

# data = pd.read_csv('./amazon-reviews.csv')   
data = pd.read_csv('./amazon-reviews.csv', header=None, names=['Review'])
pd.set_option('display.max_rows', 20)# Keep the output on one page

data

## 使用 detect_sentiment API 用于实时检测

首先，我们将使用detect_sentiment API。 DetectSentiment 操作返回一个包含检测到的情绪的对象和一个 SentimentScore 对象。

让我们检查一个纯文本示例开始。

步骤：
* 使用boto3初始化comprehend客户端
* 定义示例文本
* 调用detect_sentiment API 并将文本作为输入参数传入。 

In [ ]:
import boto3
import json

comprehend = boto3.client(service_name='comprehend', region_name=region)
                
text = "It is raining today in Seattle"

print('Calling DetectSentiment')
print(json.dumps(comprehend.detect_sentiment(Text=text, LanguageCode='en'), sort_keys=True, indent=4))
print('End of DetectSentiment\n')


现在对示例数据集使用 detect_sentiment API 并检查响应。

注意：我们只是测试了 5 条评论，然后检查输出 

In [ ]:
for index, row in data.iloc[:5].iterrows():
    print(row[0])
    print("\n")
    print(json.dumps(comprehend.detect_sentiment(Text=row[0], LanguageCode='en'), sort_keys=True, indent=4))
    print("\n")

## 使用 batch_detect_sentiment API
您最多可以在一个batch api中发送 25 篇文档进行分析。调用批处理操作的方法与调用单个文档 API 进行处理操作的方法类似。使用批处理 API 可以为您的应用程序带来更好的性能。 

In [ ]:
#We will prepare a list of the 25 review document so we can use it for batch function
rows,columns=data.shape

list_text=[] #your empty list 
for index in range(25): #iteration over the dataframe
    list_text.append(data.iat[index,0])


In [ ]:
response = comprehend.batch_detect_sentiment(
    TextList=list_text,
    LanguageCode='en'
)

print(response)

## 使用 StartSentimentDetectionJob API 进行异步批处理操作

要分析大型文档和大型文档集合，请使用 Amazon Comprehend 异步操作。

要分析文档集合，您通常执行以下步骤：

    * 将文档存储在 Amazon S3 存储桶中。

    * 开始一项或多项工作来分析文件。

    * 监控分析工作的进度。

    * 作业完成后，从 S3 存储桶中检索分析结果。

以下部分介绍了使用 Amazon Comprehend API 运行异步操作。

我们将使用 StartSentimentDetectionJob API 以检测集合中每个文档中的情绪。 

In [ ]:
# upload data to s3
s3 = boto3.resource('s3')

s3.Bucket(bucket).upload_file("amazon-reviews.csv", "{}/amazon-reviews.csv".format(prefix))

In [ ]:
#This name should match the name of the role that was created in the first lab.
role_name_base = 'AmazonComprehendServiceRoleS3FullAccess-ComprehendLabs'
prefix_random_numbers = '' #If you added random numbers to the end of the 'ComprehendLabs' prefix, put them here
if not prefix_random_numbers:
    role_name = "{}{}".format(role_name_base,prefix_random_numbers)
else:
    role_name = role_name_base
iam_client = boto3.client("iam")
response = iam_client.get_role(
    RoleName=role_name
)
comprehend_arn = response['Role']['Arn']
print("The ARN for the role is {}".format(comprehend_arn))

In [ ]:
import uuid
job_uuid = uuid.uuid1()
job_name = f"sentimentanalysis-job-{job_uuid}"
inputs3uri= bucketuri+"/amazon-reviews.csv"
asyncresponse = comprehend.start_sentiment_detection_job(
    InputDataConfig={
        'S3Uri': inputs3uri,
        'InputFormat': 'ONE_DOC_PER_LINE'
    },
    OutputDataConfig={
        'S3Uri': bucketuri,
       
    },
    DataAccessRoleArn=comprehend_arn,
    JobName=job_name,
    LanguageCode='en',
 
)

In [ ]:
# check job status
events_job_id = asyncresponse['JobId']
job = comprehend.describe_sentiment_detection_job(JobId=events_job_id)
print(job)

#### 同时也可以在控制台上查看到任务状态

In [ ]:
from time import sleep
# Get current job status
job = comprehend.describe_sentiment_detection_job(JobId=events_job_id)
print(job)
# Loop until job is completed
waited = 0
timeout_minutes = 10
while job['SentimentDetectionJobProperties']['JobStatus'] != 'COMPLETED':
    sleep(60)
    waited += 60
    assert waited//60 < timeout_minutes, "Job timed out after %d seconds." % waited
    job = comprehend.describe_sentiment_detection_job(JobId=events_job_id)

#### 该作业大约需要 6-8 分钟才能完成，您可以从作业参数中指定的输出位置下载输出。 
#### 您可以在控制台中打开 Comprehend 并在那里查看作业的详细信息。 当您有多个文档并且想要运行异步批处理时，该方法将非常有用。 